In [3]:
import zipfile
import pandas as pd
from association_alarm_id_to_lp_id import associate_alarm_id_to_lp_id

## Unzip Alarms Datasets

In [3]:
# Percorso del file ZIP da estrarre
zip_mob_file_path = "alarms datasets/mob/20230101-20240101_inpas_mob_preprocess__an__last_event__last_event__ext1.zip"
zip_tx_adsl_file_path = "alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__adsl__last_event__last_event__ext1.zip"
zip_tx_pdh_file_path = "alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__pdh__last_event__last_event__ext1.zip"
zip_tx_ptn_file_path = "alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__ptn__last_event__last_event__ext1.zip"
zip_tx_sdh_file_path = "alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__sdh__last_event__last_event__ext1.zip"

# Percorso della cartella di destinazione per l'estrazione
extract_to_folder_mob = "alarms datasets/mob"
extract_to_folder_tx = "alarms datasets/tx"

# Percorso del file ZIP da estrarre
zip_files_paths = {
    zip_mob_file_path: extract_to_folder_mob,
    zip_tx_adsl_file_path: extract_to_folder_tx,
    zip_tx_pdh_file_path: extract_to_folder_tx,
    zip_tx_ptn_file_path: extract_to_folder_tx,
    zip_tx_sdh_file_path: extract_to_folder_tx
}

# Aprire il file ZIP
for zip_file_path, extract_to_folder in zip_files_paths.items():
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Estrarre tutto il contenuto nella cartella di destinazione
        zip_ref.extractall(extract_to_folder)

## Load Datasets in a Dataframe

In [2]:
mob_alarms_df = pd.read_parquet('alarms datasets/mob/20230101-20240101_inpas_mob_preprocess__an__last_event__last_event__ext1.parquet')

In [4]:
tx_alarms_adsl_df = pd.read_parquet('alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__adsl__last_event__last_event__ext1.parquet')
tx_alarms_pdh_df = pd.read_parquet('alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__pdh__last_event__last_event__ext1.parquet')
tx_alarms_ptn_df = pd.read_parquet('alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__ptn__last_event__last_event__ext1.parquet')
tx_alarms_sdh_df = pd.read_parquet('alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__sdh__last_event__last_event__ext1.parquet')

## Clean Alarms Datasets removing old "lavori programmati" columns

In [4]:
filtered_mob_alarms_df = mob_alarms_df.drop(columns=['lp_type', 'lp_id'])

In [5]:
filtered_tx_alarms_adsl_df = tx_alarms_adsl_df.drop(columns=['lp_type', 'lp_id'])

In [6]:
filtered_tx_alarms_pdh_df = tx_alarms_pdh_df.drop(columns=['lp_type', 'lp_id'])

In [7]:
filtered_tx_alarms_ptn_df = tx_alarms_ptn_df.drop(columns=['lp_type', 'lp_id'])

In [9]:
filtered_tx_alarms_sdh_df = tx_alarms_sdh_df.drop(columns=['lp_type', 'lp_id'])

## Save Cleaned Datasets 


In [ ]:
filtered_mob_alarms_df.to_parquet('alarms datasets/mob/[filtered]20230101-20240101_inpas_mob_preprocess__an__last_event__last_event__ext1.parquet')

NameError: name 'filtered_mob_alarms_df' is not defined

In [10]:
filtered_tx_alarms_adsl_df.to_parquet('alarms datasets/tx/[filtered]20230101-20240101_inpas_tx_preprocess__adsl__last_event__last_event__ext1.parquet')

In [11]:
filtered_tx_alarms_pdh_df.to_parquet('alarms datasets/tx/[filtered]20230101-20240101_inpas_tx_preprocess__pdh__last_event__last_event__ext1.parquet')

In [12]:
filtered_tx_alarms_ptn_df.to_parquet('alarms datasets/tx/[filtered]20230101-20240101_inpas_tx_preprocess__ptn__last_event__last_event__ext1.parquet')

In [13]:
filtered_tx_alarms_sdh_df.to_parquet('alarms datasets/tx/[filtered]20230101-20240101_inpas_tx_preprocess__sdh__last_event__last_event__ext1.parquet')

## Associate Alarms with Lavori Programmati

In [14]:
lavori_programmati_df = pd.read_csv("lavori programmati/20230101_20240101_export_cac_all_preprocessed.csv")

In [ ]:
new_mob_alarms_df = associate_alarm_id_to_lp_id(filtered_mob_alarms_df, lavori_programmati_df)

In [15]:
new_tx_alarms_adsl_df = associate_alarm_id_to_lp_id(filtered_tx_alarms_adsl_df, lavori_programmati_df)

In [ ]:
new_tx_alarms_pdh_df = associate_alarm_id_to_lp_id(filtered_tx_alarms_pdh_df, lavori_programmati_df)

In [ ]:
new_tx_alarms_ptn_df = associate_alarm_id_to_lp_id(filtered_tx_alarms_ptn_df, lavori_programmati_df)

## Associate Alarms with Lavori Programmati v2

In [ ]:
filtered_mob_alarms_df = pd.read_parquet('alarms datasets/mob/[filtered]20230101-20240101_inpas_mob_preprocess__an__last_event__last_event__ext1.parquet')

In [ ]:
filtered_tx_alarms_adsl_df = pd.read_parquet('alarms datasets/tx/[filtered]20230101-20240101_inpas_tx_preprocess__adsl__last_event__last_event__ext1.parquet')

In [ ]:
filtered_tx_alarms_pdh_df = pd.read_parquet('alarms datasets/tx/[filtered]20230101-20240101_inpas_tx_preprocess__pdh__last_event__last_event__ext1.parquet')

In [ ]:
filtered_tx_alarms_ptn_df = pd.read_parquet('alarms datasets/tx/[filtered]20230101-20240101_inpas_tx_preprocess__ptn__last_event__last_event__ext1.parquet')

In [ ]:
filtered_tx_alarms_sdh_df = pd.read_parquet('alarms datasets/tx/[filtered]20230101-20240101_inpas_tx_preprocess__sdh__last_event__last_event__ext1.parquet')

In [16]:
lavori_programmati_short_df = pd.read_csv("lavori programmati/20230101_20240101_lp.csv")
lavori_programmati_short_df = lavori_programmati_short_df.iloc[:, 1:] # drop the first column

In [17]:
lavori_programmati_short_df

,alarm_id,lp_id,lp_type
0,COS1B;119303965,['3018.01/2022'],['CAVO']
1,COS1B;119303970,['3018.01/2022'],['CAVO']
2,COS1B;119303974,['3018.01/2022'],['CAVO']
3,COS1B;119303994,['3018/2022' '3018.01/2022'],['CAVO']
4,COS1B;119304000,['3018.01/2022'],['CAVO']
...,...,...,...
144034,EFMTX;6583a1f9469e8c000136ff65,['2465/2023'],['CAVO']
144035,EFMTX;6583a7fa469e8c0001370239,['2465/2023'],['CAVO']
144036,EFMTX;6583a7fa469e8c000137023b,['2465/2023'],['CAVO']
144037,EFMTX;6583b543ba43af0001748f94,['2465/2023'],['CAVO']


In [ ]:
new_mob_alarms_df = filtered_mob_alarms_df.merge(lavori_programmati_short_df, on='alarm_id', how='inner')


In [18]:
new_mob_alarms_df.to_parquet('alarms datasets/mob/[new]20230101-20240101_inpas_mob_preprocess__an__last_event__last_event__ext1.parquet')

NameError: name 'new_mob_alarms_df' is not defined

In [19]:
new_tx_alarms_adsl_df = filtered_tx_alarms_adsl_df.merge(lavori_programmati_short_df, on='alarm_id', how='inner')
new_tx_alarms_adsl_df.to_parquet('alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__adsl__last_event__last_event__ext1.parquet')

In [20]:
new_tx_alarms_pdh_df = filtered_tx_alarms_pdh_df.merge(lavori_programmati_short_df, on='alarm_id', how='inner')
new_tx_alarms_pdh_df.to_parquet('alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__pdh__last_event__last_event__ext1.parquet')

In [21]:
new_tx_alarms_ptn_df = filtered_tx_alarms_ptn_df.merge(lavori_programmati_short_df, on='alarm_id', how='inner')
new_tx_alarms_ptn_df.to_parquet('alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__ptn__last_event__last_event__ext1.parquet')

In [22]:
new_tx_alarms_sdh_df = filtered_tx_alarms_sdh_df.merge(lavori_programmati_short_df, on='alarm_id', how='inner')
new_tx_alarms_sdh_df.to_parquet('alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__sdh__last_event__last_event__ext1.parquet')

## Driectly Load New Datasets

- Use this section only if you already have [new] datasets

In [ ]:
new_tx_alarms_adsl_df = pd.read_parquet('alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__adsl__last_event__last_event__ext1_with_lp_id.parquet')

In [ ]:
new_tx_alarms_pdh_df = pd.read_parquet('alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__pdh__last_event__last_event__ext1_with_lp_id.parquet')

In [ ]:
new_tx_alarms_ptn_df = pd.read_parquet('alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__ptn__last_event__last_event__ext1_with_lp_id.parquet')

In [ ]:
new_mob_alarms_df = pd.read_parquet('alarms datasets/mob/[new]20230101-20240101_inpas_mob_preprocess__an__last_event__last_event__ext1_with_lp_id.parquet')

In [ ]:
new_tx_alarms_sdh_df = pd.read_parquet('alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__sdh__last_event__last_event__ext1_with_lp_id.parquet')

FileNotFoundError: [Errno 2] No such file or directory: 'alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__sdh__last_event__last_event__ext1_with_lp_id.parquet'

# Check New Datasets

In [ ]:
new_tx_alarms_adsl_df.dropna(subset=['lp_id'])

,alarm_id,data,first_occurrence,first_severity,last_occurrence,last_update,loc_name,make_prediction,ne_id,network,...,is_ack_or_notes,is_tt,is_abam_tt,tt_id,link_id,link_type,loc_name_a,loc_name_z,lp_id,lp_type
65481,COS4P;86787113,2023-01-10 05:06:03,2023-01-10 04:59:42,critical,2023-01-10 05:03:12,2023-01-10 05:05:08,fonzitab,False,fonzitab/adsl/1,adsl,...,False,False,False,<NA>,c6jt9cq,dmx2,NaN,NaN,[2817/2022],[CAVO]
65482,COS4P;86787114,2023-01-10 05:06:03,2023-01-10 04:59:42,critical,2023-01-10 05:03:12,2023-01-10 05:05:08,fonzitac,False,fonzitac/adsl/1,adsl,...,False,False,False,<NA>,c91v8vj,dmx2,NaN,NaN,[2817/2022],[CAVO]
65497,COS4P;86787129,2023-01-10 05:14:02,2023-01-10 05:02:15,critical,2023-01-10 05:12:31,2023-01-10 05:13:07,fonzitac,False,fonzitac/adsl/1,adsl,...,False,False,False,<NA>,c91v8vj,dmx2,NaN,NaN,[2817/2022],[CAVO]
65498,COS4P;86787122,2023-01-10 05:14:02,2023-01-10 05:02:02,critical,2023-01-10 05:12:01,2023-01-10 05:13:08,fonzitab,False,fonzitab/adsl/1,adsl,...,True,False,False,<NA>,c6jt9cq,dmx2,NaN,NaN,[2817/2022],[CAVO]
65499,COS4P;86787128,2023-01-10 05:14:02,2023-01-10 05:02:15,critical,2023-01-10 05:12:31,2023-01-10 05:13:08,fonzitab,False,fonzitab/adsl/1,adsl,...,False,False,False,<NA>,c6jt9cq,dmx2,NaN,NaN,[2817/2022],[CAVO]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9181339,EFMTX;6582ab79bfa2b40001b93ecb,2023-12-20 10:02:03,2023-12-20 09:53:13,critical,2023-12-20 09:55:13,2023-12-20 10:01:47,poaritaf081,False,poaritaf081/onuc/1,adsl,...,False,False,False,<NA>,c20e8uo,otc,NaN,NaN,[2561/2023],[APPARATI]
9181434,EFMTX;6582ab2b4c37900001c87f90,2023-12-20 10:08:02,2023-12-20 09:51:55,critical,2023-12-20 10:06:47,2023-12-20 10:06:53,poaritaf081,False,poaritaf081/onuc/1,adsl,...,False,False,False,<NA>,c20e8uo,otc,NaN,NaN,[2561/2023],[APPARATI]
9190759,EFMTX;65839325f25c6e000142ffb9,2023-12-21 03:46:03,2023-12-21 02:21:41,critical,2023-12-21 03:40:13,2023-12-21 03:45:39,catoitaa001,False,catoitaa001/onuc/1,adsl,...,False,False,False,<NA>,cv5emft,otc,NaN,NaN,[2564/2023],[CAVO]
9190760,EFMTX;65839350469e8c000136f7a3,2023-12-21 03:46:03,2023-12-21 02:22:24,critical,2023-12-21 03:40:29,2023-12-21 03:45:39,catoitaa001,False,catoitaa001/onuc/1,adsl,...,True,False,False,<NA>,cv5emft,otc,NaN,NaN,[2564/2023],[CAVO]


In [ ]:
new_tx_alarms_pdh_df.dropna(subset=['lp_id'])

,alarm_id,data,first_occurrence,first_severity,last_occurrence,last_update,loc_name,make_prediction,ne_id,network,...,is_ack_or_notes,is_tt,is_abam_tt,tt_id,link_id,link_type,loc_name_a,loc_name_z,lp_id,lp_type
32525,COS3P;66710247,2023-01-20 06:22:02,2023-01-20 04:18:05,critical,2023-01-20 06:20:29,2023-01-20 06:21:20,maiditah,False,maiditah/d3f/1,pdh,...,True,False,False,<NA>,chd2jw9,n34,NaN,NaN,[62/2023],[CAVO]
32528,COS3P;66710246,2023-01-20 06:24:02,2023-01-20 04:18:05,critical,2023-01-20 06:22:39,2023-01-20 06:23:25,maiditah,False,maiditah/d3f/1,pdh,...,True,False,False,<NA>,chd2vl2,n34,NaN,NaN,[62/2023],[CAVO]
59143,COS3P;66948725,2023-02-01 03:40:02,2023-02-01 02:03:48,critical,2023-02-01 03:38:06,2023-02-01 03:39:34,trapitan,False,trapitan/d3--/1,pdh,...,True,False,False,<NA>,c1g6qhm,482,NaN,NaN,[30.01/2023],[CAVO]
65059,COS3P;66987997,2023-02-03 04:28:02,2023-02-03 02:05:05,critical,2023-02-03 03:53:20,2023-02-03 04:26:38,romaitaw,False,romaitaw/d5f-aa/1,pdh,...,True,False,False,<NA>,cft2rl4,nm,NaN,NaN,[159/2023],[CAVO]
65060,COS3P;66987998,2023-02-03 04:28:02,2023-02-03 02:05:05,critical,2023-02-03 03:53:20,2023-02-03 04:26:38,romaitaw,False,romaitaw/d5f-aa/1,pdh,...,True,False,False,<NA>,cft2rl4,nm,NaN,NaN,[159/2023],[CAVO]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776602,EFMTX;657b976f7958460001a66dce,2023-12-15 02:56:02,2023-12-15 01:01:51,critical,2023-12-15 02:55:52,2023-12-15 02:55:59,bergitax,False,bergitax/d5f-aa/1,pdh,...,True,False,False,<NA>,ceq2gq4,nm,NaN,NaN,[2380/2023],[CAVO]
776603,EFMTX;657b976f7958460001a66dd0,2023-12-15 02:56:02,2023-12-15 01:01:51,critical,2023-12-15 02:55:52,2023-12-15 02:55:59,bergitax,False,bergitax/d5f-aa/1,pdh,...,True,False,False,<NA>,ceq2gq4,nm,NaN,NaN,[2380/2023],[CAVO]
776605,EFMTX;657b976ff651c500012ed982,2023-12-15 02:56:02,2023-12-15 01:01:51,critical,2023-12-15 02:55:40,2023-12-15 02:55:59,bresitaw,False,bresitaw/d3--/1,pdh,...,True,False,False,<NA>,ckn2nb3,482,NaN,NaN,[2380/2023],[CAVO]
776609,EFMTX;657b97707958460001a66dd2,2023-12-15 02:56:02,2023-12-15 01:01:51,minor,2023-12-15 02:55:52,2023-12-15 02:55:59,bergitax,False,bergitax/d5f-aa/1,pdh,...,True,False,False,<NA>,ceq2gq4,nm,NaN,NaN,[2380/2023],[CAVO]


In [ ]:
new_tx_alarms_ptn_df.dropna(subset=['lp_id'])

,alarm_id,data,first_occurrence,first_severity,last_occurrence,last_update,loc_name,make_prediction,ne_id,network,...,is_ack_or_notes,is_tt,is_abam_tt,tt_id,link_id,link_type,loc_name_a,loc_name_z,lp_id,lp_type
9179,COS1P;137234354,2023-01-11 04:34:02,2023-01-11 01:07:27,major,2023-01-11 04:32:39,2023-01-11 04:33:44,massital,False,massital/ptn4/1,ptn,...,False,False,False,<NA>,c6sbjfb,vc4,NaN,NaN,[3000/2022],[CAVO]
9180,COS1P;137234436,2023-01-11 04:34:02,2023-01-11 01:07:48,critical,2023-01-11 04:32:40,2023-01-11 04:33:44,fiviitae,False,fiviitae/ptn3/1,ptn,...,False,False,False,<NA>,ccmc7gc,s3t,NaN,NaN,[3000/2022],[CAVO]
9181,COS1P;137234355,2023-01-11 04:38:02,2023-01-11 01:07:27,major,2023-01-11 04:35:49,2023-01-11 04:36:50,massital,False,massital/ptn4/1,ptn,...,False,False,False,<NA>,cigj0g1,vc1,NaN,NaN,[3000/2022],[CAVO]
9182,COS1P;137234437,2023-01-11 04:38:02,2023-01-11 01:07:48,major,2023-01-11 04:35:48,2023-01-11 04:36:50,fiviitae,False,fiviitae/ptn3/1,ptn,...,False,False,False,<NA>,cigj0g1,vc1,NaN,NaN,[3000/2022],[CAVO]
9183,COS1P;137237760,2023-01-11 04:38:02,2023-01-11 04:33:35,major,2023-01-11 04:35:49,2023-01-11 04:36:50,fiviital,False,fiviital/ptn3/1,ptn,...,False,False,False,<NA>,cigj0g1,vc1,NaN,NaN,[3000/2022],[CAVO]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356528,EFMTX;65541afa743340000185fe49,2023-11-15 05:44:02,2023-11-15 02:12:26,major,2023-11-15 05:41:53,2023-11-15 05:42:15,codoitae/lodiitaj,False,codoitae/lodiitaj,ptn,...,True,False,False,<NA>,c5gs3md,vc4,codoitae,lodiitaj,[1986/2023],[CAVO]
356563,EFMTX;65541a539e8e4e00018152ec,2023-11-15 06:16:02,2023-11-15 02:09:39,critical,2023-11-15 05:39:00,2023-11-15 06:14:38,lodiitaj,False,lodiitaj/ptn4/1,ptn,...,True,False,False,<NA>,c661xma,s3t,NaN,NaN,[1986/2023],[CAVO]
371651,EFMTX;65652e9794d3a800013a7885,2023-11-28 05:30:02,2023-11-28 01:04:39,critical,2023-11-28 05:28:53,2023-11-28 05:29:08,brssital,False,brssital/ptn3/2,ptn,...,True,False,False,<NA>,c1zjq2y,s3t,NaN,NaN,[2339/2023],[CAVO]
371652,EFMTX;65656978e9c3c30001684ae2,2023-11-28 05:30:02,2023-11-28 05:15:52,critical,2023-11-28 05:28:54,2023-11-28 05:29:08,vipiitah,False,vipiitah/ptn3/1,ptn,...,False,False,False,<NA>,c1zjq2y,s3t,NaN,NaN,[2339/2023],[CAVO]


In [ ]:
new_mob_alarms_df.dropna(subset=['lp_id'])

,alarm_id,data,first_occurrence,first_severity,last_occurrence,last_update,loc_descr,make_prediction,ne_id,network,...,is_ack_or_notes,is_tt,is_abam_tt,loc_name,tt_id,link_id,loc_name_a,loc_name_z,lp_id,lp_type
473287,EFM;63bb9208a461510001d72d82,2023-01-09 05:14:02,2023-01-09 05:03:20,critical,2023-01-09 05:12:06,2023-01-09 05:13:51,cst009 timpone magara,False,8btscs009,an,...,True,False,False,castitah,<NA>,ceu6j2g,None,None,[2988/2022],[APPARATI]
473288,EFM;63bb92092aaf2c000125e980,2023-01-09 05:14:02,2023-01-09 05:03:21,critical,2023-01-09 05:13:21,2023-01-09 05:13:51,cst009 timpone magara,False,8btscs009,an,...,False,False,False,castitah,<NA>,ceu6j2g,None,None,[2988/2022],[APPARATI]
473505,EFM;63bb9457a461510001d72df4,2023-01-09 05:44:01,2023-01-09 05:13:11,major,2023-01-09 05:13:21,2023-01-09 05:17:18,cst009 timpone magara,True,8btscs009,an,...,False,False,False,castitah,<NA>,ceu6j2g,None,None,[2988/2022],[APPARATI]
473588,EFM;63bb94572aaf2c000125e9b8,2023-01-09 05:20:02,2023-01-09 05:13:11,critical,2023-01-09 05:13:21,2023-01-09 05:18:27,cst009 timpone magara,False,8btscs009,an,...,False,False,False,castitah,<NA>,ceu6j2g,None,None,[2988/2022],[APPARATI]
473589,EFM;63bb9457a461510001d72df2,2023-01-09 05:20:02,2023-01-09 05:13:11,critical,2023-01-09 05:13:21,2023-01-09 05:18:27,cst009 timpone magara,False,8btscs009,an,...,False,False,False,castitah,<NA>,ceu6j2g,None,None,[2988/2022],[APPARATI]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27453669,EFM;65823deab64e580001615826,2023-12-20 16:36:03,2023-12-20 02:05:46,critical,2023-12-20 16:35:44,2023-12-20 16:35:49,vtt02f montalto marina sud,False,gbtsvt02f,an,...,False,False,False,tarqitcm,<NA>,cawdiec,None,None,[2464/2023],[CAVO]
27499998,EFM;65823dea9da652000141075c,2023-12-21 08:52:03,2023-12-20 02:05:46,critical,2023-12-20 16:35:44,2023-12-21 08:50:02,vtt049 pescia romana,False,gbtsvt049,an,...,False,True,False,tarqitbd,inpfl1001624631,cuy2dh3,None,None,[2464/2023],[CAVO]
27499999,EFM;65823dea9da652000141075f,2023-12-21 08:52:03,2023-12-20 02:05:46,critical,2023-12-20 16:34:17,2023-12-21 08:50:02,vtt049 pescia romana,False,gbtsvt049,an,...,True,True,False,tarqitbd,inpfl1001624631,cuy2dh3,None,None,[2464/2023],[CAVO]
27500000,EFM;65823deab64e580001615823,2023-12-21 08:52:03,2023-12-20 02:05:46,critical,2023-12-20 16:35:44,2023-12-21 08:50:02,vtt02f montalto marina sud,False,gbtsvt02f,an,...,False,True,False,tarqitcm,inpfl1001624629,cawdiec,None,None,[2464/2023],[CAVO]
